In [2]:
import sys
from pathlib import Path

# Add the project root directory to Python path
project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [3]:
import json
from tqdm import tqdm
import random
import pandas as pd

from config import DATA_PATH, SYSTEM_PROMPT, TEMPLATE_TYPES
from utils.model_interactions import generate_response_openai

In [7]:
data_path = DATA_PATH / "examples_teacher_tasks.jsonl"
with open(data_path, encoding="utf-8") as f:
    example_tasks = [json.loads(line) for line in f]

In [8]:
generated = []
for task in tqdm(example_tasks, desc="Generating samples"):
    reply = generate_response_openai(SYSTEM_PROMPT, task["instruction"])
    task_out = {
        "system": SYSTEM_PROMPT,
        "instruction": task["instruction"],
        "response": reply,
        "metadata": {
            "type": task["type"],
            "level": task["level"]
        }
    }
    generated.append(task_out)

Generating samples: 100%|██████████| 5/5 [01:14<00:00, 14.84s/it]


In [9]:
train_path = DATA_PATH / "train_teacher_mixed.jsonl"
with open(train_path, "w", encoding="utf-8") as f:
    for item in generated:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print("✅ Dataset saved: train_teacher_mixed.jsonl")

✅ Dataset saved: train_teacher_mixed.jsonl


In [10]:
df = pd.DataFrame(generated)
df_review = df[["instruction", "response"]]
df_review.columns = ["User Prompt", "Model Response"]

# Save as Excel and DOCX
xlsx_path = DATA_PATH / "train_teacher_mixed.xlsx"
df_review.to_excel(xlsx_path, index=False)

import docx
from docx.shared import Pt

doc = docx.Document()
doc.add_heading("AI Responses for Review", 0)

for i, row in df_review.iterrows():
    doc.add_paragraph(f"{i+1}. Prompt:", style='Heading 2')
    doc.add_paragraph(row["User Prompt"])
    doc.add_paragraph("Response:", style='Heading 2')
    doc.add_paragraph(row["Model Response"])
    doc.add_paragraph("\n")

docx_path = DATA_PATH / "train_teacher_mixed.docx"
doc.save(docx_path)

In [6]:
train_path = DATA_PATH / "examples_teacher_tasks.jsonl"
with open(train_path, encoding="utf-8") as f:
    raw_tasks = [json.loads(line) for line in f if line.strip()]

# === 4. Распределение по типам ===
TASKS_PER_TYPE = 100  # Итого 5 типов × 100 = 500 заданий

selected_tasks = []
for template_type in TEMPLATE_TYPES:
    typed_tasks = [t for t in raw_tasks if t["type"] == template_type]
    if not typed_tasks:
        continue
    # если меньше, чем нужно — дублируем с перемешиванием
    if len(typed_tasks) < TASKS_PER_TYPE:
        extended = typed_tasks * (TASKS_PER_TYPE // len(typed_tasks) + 1)
        typed_tasks = extended[:TASKS_PER_TYPE]
    selected_tasks.extend(random.sample(typed_tasks, TASKS_PER_TYPE))

In [7]:
# === 5. Генерация примеров ===
generated = []
for task in tqdm(selected_tasks, desc="Generating 500 samples"):
    try:
        reply = generate_response_openai(SYSTEM_PROMPT, task["instruction"])
        task_out = {
            "system": SYSTEM_PROMPT,
            "instruction": task["instruction"],
            "response": reply,
            "metadata": {
                "type": task["type"],
                "level": task["level"]
            }
        }
        generated.append(task_out)
    except Exception as e:
        print("❌ Error:", e)

# === 6. Сохранение ===
train_path = DATA_PATH / "train_teacher_mixed_500.jsonl"
with open(train_path, "w", encoding="utf-8") as f:
    for item in generated:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print("✅ Dataset saved: train_teacher_mixed_500.jsonl with", len(generated), "samples")

Generating 500 samples:  53%|█████▎    | 263/500 [1:13:17<58:44, 14.87s/it]  

❌ Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Generating 500 samples:  53%|█████▎    | 264/500 [1:13:19<43:20, 11.02s/it]

❌ Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Generating 500 samples:  53%|█████▎    | 265/500 [1:13:21<32:23,  8.27s/it]

...

❌ Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Generating 500 samples: 100%|█████████▉| 499/500 [1:20:25<00:01,  1.77s/it]

❌ Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


Generating 500 samples: 100%|██████████| 500/500 [1:20:27<00:00,  9.65s/it]

❌ Error: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
✅ Dataset saved: train_teacher_mixed_500.jsonl with 262 samples


In [5]:
# Пути к файлам
path1 = DATA_PATH / "Samples ChatGPT" / "generated_dialog_analysis_100.jsonl"
path2 = DATA_PATH / "Samples ChatGPT" / "generated_fill_in_the_blank_100.jsonl"
path3 = DATA_PATH / "Samples ChatGPT" / "generated_fill_in_the_blank_diverse_100.jsonl"
path4 = DATA_PATH / "Samples ChatGPT" / "generated_grammar_explanation_38.jsonl"
path5 = DATA_PATH / "Samples ChatGPT" / "generated_grammar_mixed_263.jsonl"
output_path = DATA_PATH / "Samples ChatGPT" / "train_teacher_full_ChatGPT.jsonl"

# Загрузка
all_data = []
for path in [path1, path2, path3, path4, path5]:
    with path.open(encoding="utf-8") as f:
        for line in f:
            if line.strip():
                all_data.append(json.loads(line))

print(f"📦 Объединено {len(all_data)} примеров из двух источников")

# Сохранение
with output_path.open("w", encoding="utf-8") as f:
    for item in all_data:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Финальный датасет сохранён в: {output_path.name}")

import docx
from docx.shared import Pt



📦 Объединено 600 примеров из двух источников
✅ Финальный датасет сохранён в: train_teacher_full_ChatGPT.jsonl


In [7]:
import docx
from docx.shared import Pt

path = DATA_PATH / "Samples ChatGPT" / "train_teacher_deduplicated.jsonl"

samples = []
with path.open(encoding="utf-8") as f:
    for line in f:
        if line.strip():
            samples.append(json.loads(line))

df = pd.DataFrame(samples)
df_review = df[["instruction", "response"]]
df_review.columns = ["User Prompt", "Model Response"]

doc = docx.Document()
doc.add_heading("AI Responses for Review", 0)

for i, row in df_review.iterrows():
    doc.add_paragraph(f"{i+1}. Prompt:", style='Heading 2')
    doc.add_paragraph(row["User Prompt"])
    doc.add_paragraph("Response:", style='Heading 2')
    doc.add_paragraph(row["Model Response"])
    doc.add_paragraph("\n")

docx_path = DATA_PATH / "Samples ChatGPT" / "train_teacher_deduplicated.docx"
doc.save(str(docx_path))